### Defined constants

In [1]:
import os
import numpy as np
import process_traces
traces_to_load = 0
B8 = 8
B16 = 16
B256 = 256
B8_MASK = 0xff
SIZE = 32


def tqdm_sub(x, *args, **kwargs):
    return x


try:
    from tqdm.notebook import tqdm
except ImportError:
    tqdm = tqdm_sub
    tqdm.write = print

sbox = [
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
]

inv_sbox = [
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D
]

HW = [
    0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4,
    1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5,
    1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5,
    2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6,
    1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5,
    2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6,
    2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6,
    3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7,
    1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5,
    2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6,
    2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6,
    3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7,
    2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6,
    3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7,
    3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7,
    4, 5, 5, 6, 5, 6, 6, 7, 5, 6, 6, 7, 6, 7, 7, 8
]


def apply_sbox(idx):
    return sbox[idx]


def apply_inv_sbox(idx):
    return inv_sbox[idx]


assert len(sbox) == B16*B16
assert len(inv_sbox) == B16*B16


### load in data

In [2]:
path = r'D:\OneDrive - nyu.edu\temp\data\meas2'

In [ ]:
# load in traces
if os.path.exists(f'{path}/plain_texts_bytes.npy') and os.path.exists(f'{path}/trace.npy'):
    print("loading saved data...")
    plain_texts_bytes = np.load(f'{path}/plain_texts_bytes.npy')
    traces = np.load(f'{path}/trace.npy')
else:
    print("loading traces...")
    plain_texts, traces = process_traces.get_power_trace(
        num_of_traces=0, path=path, VCC=5.25, keep_percent=0.5)

    plain_texts_bytes = []
    for i, c in enumerate(plain_texts):
        ci = []
        for j in range(int(SIZE/2)):
            ci.append(c & B8_MASK)
            c >>= B8
        ci.reverse()
        plain_texts_bytes.append(ci)

    np.save(f'{path}/plain_texts_bytes.npy', plain_texts_bytes)
    np.save(f'{path}/trace.npy', traces)
    print("saved traces and plain texts")

trace_count = len(traces)
trace_length = len(traces[0])
assert (trace_count == len(plain_texts_bytes))


## DPA

In [ ]:
save_dir = 'img_diff'
save_dir = os.path.join(path, save_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

print("start DPA...")
for bit_use in tqdm(range(B8)):
    # guess the key by bytes, from the higher byte to the lower byte
    guess_key = []
    # Mean delta accumulation of traces, just for visualization
    mean_delta_accu_visualization = []
    last_guess_key = ''
    for byte_i in tqdm(range(B16), desc=f"Use bit {bit_use}", leave=False):
        max_diffs = np.zeros(256)
        mean_delta_accu_visualization.append([])
        for key_guess in tqdm(range(B256), leave=False, desc=f"last key: {last_guess_key}, guessing byte {B16 - byte_i}"):
            # the traces will be sum up into those two bins
            guess_zero = np.zeros_like(traces[0])
            guess_one = np.zeros_like(traces[0])
            zero_count = 0
            one_count = 0
            for i in range(trace_count):
                bit = (apply_sbox(
                    plain_texts_bytes[i][byte_i] ^ key_guess) >> bit_use) & 1
                if bit == 0:
                    guess_zero += traces[i]
                    zero_count += 1
                else:
                    guess_one += traces[i]
                    one_count += 1
            if zero_count == 0:
                # TODO: might not be the best way to handle this
                diff = guess_one/one_count
            elif one_count == 0:
                # TODO: might not be the best way to handle this
                diff = guess_zero/zero_count
            else:
                diff = guess_zero/zero_count - guess_one/one_count
            diff = np.abs(diff)
            max_diffs[key_guess] = np.max(np.abs(diff))
            mean_delta_accu_visualization[byte_i].append(diff)
        # find the guess_key_i with max difference
        # note: the return type of np.argmax is np.int64, which cause trouble when
        # interpreting the result as a byte
        guess_key.append(int(np.argmax(max_diffs)))
        last_guess_key = f"{guess_key[-1]:02x}"

    res = 0
    for key_guess in guess_key:
        res += key_guess
        res <<= B8
    res >>= B8

    tqdm.write(f"Round {bit_use}: the guessed key is: {res:032x}")
    np.save(f"{save_dir}/diff_visualization_{bit_use}.npy",
            np.array(mean_delta_accu_visualization))

## CPA

In [ ]:
save_dir = 'img_corr'
save_dir = os.path.join(path, save_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
print("start CPA...")
for const_use in tqdm(range(B256), desc="const use"):
    key = []
    mean_t = np.mean(traces, axis=0)
    # Mean delta accumulation of traces, just for visualization
    correlation_on_trace_visualization = []
    last_guess_key = ''
    for byte_i in tqdm(range(B16), desc=f"const use: {const_use}", leave=False):
        max_cpa = [0]*B256
        correlation_on_trace_visualization.append([])

        for key_guess in tqdm(range(B256), desc=f"last key: {last_guess_key}", leave=False):
            power_mode = np.zeros(trace_count)
            numerator = np.zeros(trace_length)
            denominator_model = np.zeros(trace_length)
            denominator_measured = np.zeros(trace_length)

            for trace_index in range(trace_count):
                vij = apply_sbox(
                    plain_texts_bytes[trace_index][byte_i] ^ key_guess)
                power_mode[trace_index] = HW[vij ^ const_use]

            mean_h = np.mean(power_mode)

            for trace_index in range(trace_count):
                # h - h_bar
                h_diff = power_mode[trace_index] - mean_h
                # t - t_bar
                t_diff = traces[trace_index][:] - mean_t
                # (h - h_bar)(t - t_bar)
                numerator += h_diff * t_diff
                # (h - h_bar)^2
                denominator_model += h_diff ** 2
                # (t - t_bar)^2
                denominator_measured += t_diff ** 2
            correlation_on_trace = numerator / \
                np.sqrt(denominator_model*denominator_measured)
            max_cpa[key_guess] = max(abs(correlation_on_trace))
            correlation_on_trace_visualization[byte_i].append(
                correlation_on_trace)
        key.append(np.argmax(max_cpa))
        last_guess_key = f"{key[-1]:02x}"

    res = 0
    for key_guess in key:
        res += key_guess
        res <<= B8
    res >>= B8

    tqdm.write(f"Round {const_use}: the guessed key is: {res:032x}")
    np.save(f"{save_dir}/corr_visualization_{const_use}.npy",
            np.array(correlation_on_trace_visualization))
